# Creating PCs and GRM

Author: Jose Jaime Martinez-Magana

Day: 28 March 2023

This script was developed to estimate principal component analysis and the GRM using admixture for the Tractor - Mix model for in the Yale HPC McCleary -  cluster. Using the Yale Penn 2 cohort.

In [ ]:
# move to the working directory
cd /vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix

# create the directories to store the results of the pca and grm files
mkdir pca_grm

# create the directories to store the plink files
mkdir plink_files
# create the following directories inside the plink file directory
mkdir plink_files/prunne_snps plink_files/first_plink plink_files/prunned_plink

# create the directories to store the scripts for pca_grm
mkdir scripts/pca_grm

# move to the directory
cd scripts/pca_grm

In [ ]:
# we will transform the vcf files to plink files and prunned the plink files, named vcfto_plink_prunned.sh

# start script
#!/bin/bash
#SBATCH --job-name=yale_penn_vftto_plink_files_prunned
#SBATCH --out="slurm-%j.out"
#SBATCH --time=18:00:00
#SBATCH --nodes=1 --ntasks=1 --cpus-per-task=4
#SBATCH --mem-per-cpu=90G
#SBATCH --mail-type=ALL
####################################################################################
# script for building plink files from of Yale Penn 2 cohort vcfs files
# day: 28 March 2023
# analyzer: Jose Jaime Martinez-Magana - jjm262
# cluster: McCleary - HPC Yale
####################################################################################
# This script uses the Yale Penn cohort filtered by AFR american ancestry higher
# than 50%
# the data for input to PCAiR
####################################################################################
# set parameters
# set plink files paths
yp_pf='/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/databases/genotype/phased_vcfs'
# set output paths
yp_op='/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/plink_files'
# plink path
plink='/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/scripts/environment/extdata/plink/plink'
####################################################################################
# run analysis
## build plink files for yale penn 2
for chr in {1..22}
do
$plink --vcf "${yp_pf}"/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.vcf \
--make-bed \
--keep-allele-order \
--out "${yp_op}"/first_plink/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.plink \
--id-delim '_' \
--threads $SLURM_CPUS_PER_TASK
done

# after we will prunned the files 
# running ld prunning
for chr in {1..22}
do
$plink --bfile "${yp_op}"/first_plink/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.plink \
--maf 0.05 \
--hwe 0.000005 \
--indep 50 5 2 \
--threads $SLURM_CPUS_PER_TASK \
--out "${yp_op}"/prunne_snps/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.plink_prune_snps
done

# removing snps after ld prunning
for chr in {1..22}
do
$plink --bfile "${yp_op}"/first_plink/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.plink \
--extract "${yp_op}"/prunne_snps/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.plink_prune_snps.prune.in \
--make-bed \
--keep-allele-order \
--threads $SLURM_CPUS_PER_TASK \
--out "${yp_op}"/prunned_plink/GWGO_GWCIDR.1kg_phase3_v5.chr"${chr}".dose.rsids.filtered.nodup.phased.shapeit.plink_prunned_v03282023
done
# end script
# run the previous script with the next command, for this your server should have a slurm handler
# if not run your script with your handler
sbatch vcfto_plink_prunned.sh

In [ ]:
# merging plink files
# requesting computational resources
salloc -t 2:00:00 --mem=8G

# merging plink files
cd /vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/scripts/pca_grm
yp_op='/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/plink_files/prunned_plink'
find "${yp_op}" | egrep bim | sed 's/.bim//g' > plink_files_merged.list
# remove choromosome one manually from plink_files_merged.list
# merge with plink
# plink path
plink='/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/scripts/environment/extdata/plink/plink'
${plink} --bfile "${yp_op}"/GWGO_GWCIDR.1kg_phase3_v5.chr1.dose.rsids.filtered.nodup.phased.shapeit.plink_prunned_v03282023 \
--merge-list plink_files_merged.list \
--out "${yp_op}"/GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.plink_prunned_v03292023

In [ ]:
# create directory to store the pca and grm object


# creating gds object
# script named: create_gds.Rscript

#!/usr/bin/env Rscript --vanilla --slave
####################################################################################
# script for creating PCs with relationship with PCAiR
# day: 13 March 2023
# author: Jose Jaime Martinez-Magana
####################################################################################
# This script uses a plink file to create a GDS file for input to PCAiR
####################################################################################
# set parameters
# this function uses the library optparse to add arguments to the script
# adding arguments to the script
library(optparse) 
option_list = list(
    make_option(c("--plinkfile"), type="character", default=NULL,
                help="path to the directory storing the plink files, without plink extensions *bed/*bim/*fam", metavar="character"),
    make_option(c("--out"), type="character", default=NULL,
                help="output file name for GDS file, use complete path, example /data/analsyis/analysis.gds", metavar="character")
);

opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

if (is.null(opt$plinkfile)){
    print_help(opt_parser)
    stop("At least one argument must be supplied (input file)", call.=FALSE)
}

###################################################################################
# load libraries
library(GENESIS)
library(GWASTools)
library(SNPRelate)

# create list of files
bed_f=paste0(opt$plinkfile,".bed",sep="")
bim_f=paste0(opt$plinkfile,".bim",sep="")
fam_f=paste0(opt$plinkfile,".fam",sep="")

# create gds output files
snpgdsBED2GDS(bed.fn=bed_f,
              bim.fn=bim_f,
              fam.fn=fam_f,
              family=TRUE,
              out.gdsfn=opt$out)
# end of script

In [ ]:
# script for running the create_gds.Rscript
# name of script create_gds.sh

#!/bin/bash
#SBATCH --job-name=yale_penn_build_gds_files
#SBATCH --out="slurm-%j.out"
#SBATCH --time=10:00:00
#SBATCH --nodes=1 --ntasks=1 --cpus-per-task=4
#SBATCH --mem-per-cpu=90G
#SBATCH --mail-type=ALL
####################################################################################
# script for constructing GDS files
# day: 20 March 2023
# analyzer: Jose Jaime Martinez-Magana - jjm262
# cluster: Grace - HPC Yale
####################################################################################
# This script uses the R script to generate GDS files for LD prunned Yale Penn 
# plink files
####################################################################################
# load miniconda
module load miniconda
# activate environment
conda activate genesis
# run R script for creating GDS files
Rscript="/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/scripts/pca_grm/create_gds.Rscript"
Rscript $Rscript \
--plinkfile=/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/plink_files/prunned_plink/GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.plink_prunned_v03292023 \
--out=/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/pca_grm/GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.gds_prunned_v03292023.gds

# run script
sbatch create_gds.sh

In [ ]:
# developing the script for PCA and GRM with PCAiR and PCRelate
# name of script: create_grm.Rscript

#!/usr/bin/env Rscript --vanilla --slave
####################################################################################
# R script for creating GRM and PCs
# day: 29 March 2023
# author: Jose Jaime Martinez-Magana
####################################################################################
# This script uses a gds file to estimate PCs and GRM using GENESIS
####################################################################################
# set parameters
# this function uses the library optparse to add arguments to the script
# adding arguments to the script
library(optparse) 
option_list = list(
    make_option(c("--gdsfile"), type="character", default=NULL,
                help="path to gds file", metavar="character"),
    make_option(c("--out"), type="character", default=NULL,
                help="output path for PCAiR objects, use a complete directory path and a potential name of files. Example: /home/user/out_pca", metavar="character")
);

opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

if (is.null(opt$gdsfile)){
  print_help(opt_parser)
  stop("At least one argument must be supplied (input file)", call.=FALSE)
}

###################################################################################
# load libraries
library(GENESIS)
library(GWASTools)
library(SNPRelate)
library(SeqArray)
library(parallel)
library(BiocParallel)
library(stringi);

# stablishing a seed
set.seed(1000)
# stablishing cores
cores=detectCores()

# create function for LD prunning
ld_prun=function(gds){
snpset=snpgdsLDpruning(gds,
                       method="corr",
                       slide.max.bp=10e6,
                       ld.threshold=sqrt(0.1),
                       maf=0.01,
                       missing.rate=0.01,
                       verbose=TRUE, num.thread=cores);
    pruned=unlist(snpset,
                  use.names=FALSE);
return(pruned)
}


# create function for KING-robust analysis
king_mat=function(gds){
samp.id=read.gdsn(index.gdsn(gds, "sample.id"))
ibd.robust=snpgdsIBDKING(gds, sample.id=samp.id, family.id=NULL, maf=0.01,missing.rate=0.01,num.thread=cores)
return(ibd.robust)
}


# create function for PCAiR
pcair_r=function(gds_geno, pruned, KINGmat){
pcair=pcair(gds_geno, snp.include=pruned,
            kinobj=KINGmat, divobj=KINGmat)
return(pcair)
}

# create function for PCAiR with sample filter
pcair_r_fil=function(gds_geno, pruned, KINGmat, sample_list){
    pcair=pcair(gds_geno, snp.include=pruned,
                sample.include=sample_list,
                kinobj=KINGmat, divobj=KINGmat)
                return(pcair)
                }


# create function to match the sample ID in the GDS with the sample IDs of the supplied sampleID filter
filter_samples=function(gds_samples, incl_samples){
    matches=unique(grep(paste(incl_samples$SampleID,collapse="|"),
                        gds_samples, value=TRUE))
    return(matches)
}


# run analysis for pcs with relationships
# open the gds object
gds=snpgdsOpen(opt$gdsfile);
# LD prunning
pruned=ld_prun(gds)
# build KING matrix
KINGmat=king_mat(gds)
# adjust KING matrix
KINGmat_m=KINGmat$kinship
# add sampleIDs to colnames y row names
colnames(KINGmat_m)=KINGmat$sample.id
rownames(KINGmat_m)=KINGmat$sample.id
# get samples in gds
gds_samples=read.gdsn(index.gdsn(gds, "sample.id"))
# close the gds object
snpgdsClose(gds)

# read the gds object
gds=GdsGenotypeReader(filename=opt$gdsfile)

# create a GenotypeData class object
gds_geno=GenotypeData(gds)

# run PCAiR
PCair=pcair_r(gds_geno, pruned, KINGmat_m)
PCairpart=pcairPartition(kinobj = KINGmat_m, divobj = KINGmat_m)

# PC relate calculation 
gdsData=GenotypeBlockIterator(gds_geno, snpInclude=pruned)
PCrelate=pcrelate(gdsData,
                  pcs=PCair$vectors[,1:2],
                  training.set=PCairpart$unrels,
                  BPPARAM=BiocParallel::SerialParam())

# making GRM
grm=pcrelateToMatrix(PCrelate)
# making sparce matrix
grm[grm<0.05]=0
grm_sparse=as.matrix(grm, sparse = TRUE)

# save file
out=list()
out$PCair=PCair
out$PCairpart=PCairpart
out$KINGmat=KINGmat_m
out$pruned=pruned
out$PCrelate=PCrelate
out$grm=grm
out$grm_sparse=grm_sparse
saveRDS(file=paste0(opt$out,".rds",""),out)

In [ ]:
# script for running the pca and grm cosntruction
# name of script: create_grm.sh

#!/bin/bash
#SBATCH --job-name=yale_penn_build_grm_pcair
#SBATCH --out="slurm-%j.out"
#SBATCH --time=23:00:00
#SBATCH --nodes=1 --ntasks=1 --cpus-per-task=8
#SBATCH --mem-per-cpu=72G
#SBATCH --mail-type=ALL
#SBATCH --partition=day
####################################################################################
# script for constructing GDS files
# day: 29 March 2023
# analyzer: Jose Jaime Martinez-Magana - jjm262
# cluster: Grace - HPC Yale
####################################################################################
# This script uses the R script to generate GDS files for LD prunned Yale Penn 
# plink files
####################################################################################
# load miniconda
module load miniconda
# activate environment
conda activate tractor_mix
# run R script for creating GDS files
Rscript="/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/scripts/pca_grm/create_grm.Rscript"
Rscript $Rscript \
--gdsfile=/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/pca_grm/GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.gds_prunned_v03292023.gds \
--out=/vast/palmer/scratch/montalvo-ortiz/jjm262/genomics/yalepenn/tractor_mix/pca_grm/GWGO_GWCIDR.1kg_phase3_v5.chrmerged.dose.rsids.filtered.nodup.phased.shapeit.gds_prunned_grm_pca_v03292023

# running script
sbatch create_grm.sh